In [ ]:
#참고링크 https://github.com/allenai/bilm-tf
#참고링크 https://appliedmachinelearning.wordpress.com/2019/11/30/training-elmo-from-scratch-on-custom-data-set-for-generating-embeddings-tensorflow/

In [ ]:
# #cpu 사용
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# 패키지 임포트

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
import string
import nltk
import pickle
import json
sys.path.append('C:/Users/User/Desktop/AG News')
import tensorflow as tf
import scipy.spatial.distance as ds
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
from collections import Counter
from data_preprocessing import *

C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\frame

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

# 전처리

In [3]:
train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)

In [4]:
sampled_df = train_df.groupby("Class Index").apply(lambda x: x.sample(5000, random_state=10))

In [5]:
sampled_df['text']=sampled_df['text'].apply(clean_text)

In [6]:
sampled_df = sampled_df.reset_index(drop=True)

In [7]:
train_df = sampled_df.groupby("Class Index").apply(lambda x: x.sample(4000, random_state=10))
train_idx = [x[1] for x in train_df.index]
test_df = sampled_df.drop(train_idx)

In [8]:
x_train=list(train_df['text'])
y_train=list(train_df['Class Index'])
x_test=list(test_df['text'])
y_test=list(test_df['Class Index'])

In [9]:
to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

# 사전 학습 ELMo 불러오기

In [11]:
#사전 학습된 모델이 있는 디렉토리
datadir = 'elmo_model(ag256)'
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir, 'options.json')
weight_file = os.path.join(datadir, 'swb_weights.hdf5')
 
batcher = Batcher(vocab_file, 50)
 
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
 
bilm = BidirectionalLanguageModel(options_file, weight_file)
 
context_embeddings_op = bilm(context_character_ids)
 
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)

#테스트 임베딩
raw_context = ['Technology has advanced so much in new scientific world',
                'My child participated in fancy dress competition',
                'Fashion industry has seen tremendous growth in new designs']
 
tokenized_context = [sentence.split() for sentence in raw_context]
print(tokenized_context)

USING SKIP CONNECTIONS
[['Technology', 'has', 'advanced', 'so', 'much', 'in', 'new', 'scientific', 'world'], ['My', 'child', 'participated', 'in', 'fancy', 'dress', 'competition'], ['Fashion', 'industry', 'has', 'seen', 'tremendous', 'growth', 'in', 'new', 'designs']]


In [12]:
with tf.Session() as sess:    
    sess.run(tf.global_variables_initializer())     
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
    elmo_context_input_ = sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_character_ids: context_ids}
    )
 
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (3, 11, 50)
Shape of generated embeddings =  (3, 9, 256)


In [13]:
euc_dist_bet_tech_computer = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[0,0,:])
euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])
cos_dist_bet_tech_computer = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[0,0,:])
cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\nDress-Technology = ",np.round(euc_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(euc_dist_bet_computer_fashion,2))
print("\n\nCosine Distance Comparison - ")
print("\nDress-Technology = ",np.round(cos_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(cos_dist_bet_computer_fashion,2))

Euclidean Distance Comparison - 

Dress-Technology =  12.98 
Dress-Fashion =  12.22


Cosine Distance Comparison - 

Dress-Technology =  0.8 
Dress-Fashion =  0.71


임베딩

In [14]:
#메모리 오류로 임베딩 되지 않는 문서의 인덱스를 찾기위한 리스트
except_lst=[]

In [ ]:
for i in range(16000):
    try:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_train[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        np.save('./elmo_embedding/train(ag256)/doc%d'%i,elmo_context_input_[0])
    except:
        except_lst.append(i)

Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 21, 256)


In [21]:
except_lst

[]

In [19]:
except_test_lst=[]

In [20]:
for i in range(len(x_test)):
    try:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_test[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        np.save('./elmo_embedding/test(ag256)/test%d'%i,elmo_context_input_[0])
    except:
        except_test_lst.append(i)

Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (

...

Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 23, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 44, 256)


In [21]:
except_test_lst

[]